# Starting point for Spark on Google Cloud

In [1]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

## Step 2.3

In [4]:
# TODO: read files, load graph_sdf, etc.
c2a_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ')\
    .load("gs://fix-bucket/notebooks/sx-stackoverflow-c2a.txt")\
    .selectExpr("cast(_c0 as int) as from_node", "cast(_c1 as int) as to_node")
    
    
    
c2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ')\
    .load("gs://fix-bucket/notebooks/sx-stackoverflow-c2q.txt")\
    .selectExpr("cast(_c0 as int) as from_node", "cast(_c1 as int) as to_node")
    
a2q_sdf = spark.read.format("com.databricks.spark.csv").option("delimiter", ' ')\
    .load("gs://fix-bucket/notebooks/sx-stackoverflow-a2q.txt")\
    .selectExpr("cast(_c0 as int) as from_node", "cast(_c1 as int) as to_node")
    
graph_sdf = c2a_sdf.union(c2q_sdf).union(a2q_sdf)

In [5]:
# def transitive_closure(graph_sdf, origins_sdf, depth):
#     graph_sdf.cache()
#     graph_sdf.createOrReplaceTempView('graph')
#     origins_sdf.createOrReplaceTempView('origins')
#     result = spark.sql('SELECT * from origins')
#     result.createOrReplaceTempView('result')
#     frontiers = spark.sql('SELECT * from origins')
    
    
#     for i in range(0, depth):
#         frontiers.createOrReplaceTempView('frontiers')
#         frontiers.cache()
#         if (len(frontiers.take(1))==0):
#             break
#         frontiers = spark.sql('SELECT DISTINCT graph.to_node AS node'\
#         ' FROM frontiers LEFT JOIN graph ON frontiers.node=graph.from_node'\
#         ' WHERE graph.to_node NOT IN (SELECT * FROM result)').cache()
#         frontiers.createOrReplaceTempView('frontiers')
#         result = spark.sql('SELECT * from result UNION ALL SELECT * FROM frontiers').cache()
#         result.createOrReplaceTempView('result')
    
    
#     return result

In [10]:
def transitive_closure2(graph_sdf, origins_sdf, depth):
    

        
    graph_sdf.cache()
    graph_sdf.createOrReplaceTempView('graph')
    origins_sdf.createOrReplaceTempView('origins')
    result = spark.sql('SELECT * from origins')
    result.createOrReplaceTempView('result')
    frontiers = spark.sql('SELECT * from origins')
    
    if (depth==0):
            return spark.sql('SELECT node from result LIMIT 0')
    
    
    for i in range(0, depth-1):
        frontiers.createOrReplaceTempView('frontiers')
        frontiers.cache()
        if (len(frontiers.take(1))==0):
            break
        frontiers = spark.sql('SELECT DISTINCT to_node AS node'\
        ' FROM (SELECT graph.from_node, graph.to_node FROM frontiers LEFT JOIN graph'\
        ' ON frontiers.node=graph.from_node) as x'\
        ' ANTI JOIN result on x.to_node=result.node').cache()
        frontiers.createOrReplaceTempView('frontiers')
        result = spark.sql('SELECT * from result UNION ALL SELECT * FROM frontiers').cache()
        result.createOrReplaceTempView('result')
    
    
    return spark.sql('SELECT DISTINCT node FROM result')
    

In [6]:
# Compute nodes_sdf
graph_sdf.createOrReplaceTempView('graph')
nodes_sdf = spark.sql('SELECT DISTINCT from_node as node FROM graph WHERE from_node < 8').cache()
nodes_sdf.show()

+----+
|node|
+----+
|   1|
|   3|
|   5|
|   4|
|   2|
+----+



In [8]:
reachable_sdf = transitive_closure2(graph_sdf, nodes_sdf, 3).cache()

## Step 2.3 Results

In [9]:
reachable_sdf.count()

246212

In [11]:
reachable_sdf.show()

+------+
|  node|
+------+
| 17389|
|179115|
|408870|
|   148|
|   463|
|  1088|
|  1238|
|  1580|
|  1591|
|  1959|
|  2122|
|  2659|
|  3175|
|  4101|
|  4900|
|  6336|
|  6466|
|  7754|
|  8638|
|  9465|
+------+
only showing top 20 rows

